In [13]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import ipyvolume as ipv

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget
import numpy as np

# file = "./group_1401.xyz"
file ="../../data/unsampled/xyt/points/noise/group_5006.xyz"
# file ="../../data/unsampled/xyt/points/mixed/group_103.xyz"
# output_path="your_path_to_output_folder/"

In [26]:
def normalise(pcd):
    """
    Computes the normals of a point cloud. Normals are oriented 
    with respect to the input point cloud if normals exist.
    Next, converts float64 numpy array of shape (n, 3) to Open3D format.
    Computed points are returned
    """
    
    radius = 0.1
    max_nn = 300

    try:
        pcd.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius,
                                                                                 max_nn),
                             fast_normal_computation=False)
        pcd.normals = o3d.utility.Vector3dVector(np.asarray(pcd.normals))
        return pcd
    except RuntimeError:
        print("Normals were not computed!")
        

def initialise_pointcloud(point_cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    normalised_pcd = normalise(pcd)
    pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
    
    return pcd

In [22]:
point_cloud= np.loadtxt(file)
pcd = initialise_pointcloud(point_cloud)

In [ ]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3.5 * avg_dist
    


In [ ]:
radius

In [ ]:
# BPA
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(cl,o3d.utility.DoubleVector([radius,
                                                                                                         radius * 2]))
# dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

bpa_mesh.paint_uniform_color([7, 3, 4])
o3d.visualization.draw_geometries([bpa_mesh])

In [27]:
# POISSON

def poisson_mesh(pcd):
    """
    Computes and returns a triangle mesh from a oriented PointCloud pcd. 
    It implements the Screened Poisson Reconstruction proposed in 
    Kazhdan and Hoppe, "Screened Poisson Surface Reconstruction", 2013. 
    See https://github.com/mkazhdan/PoissonRecon
    Returns a cropped geometry based on an axis-aligned bounding box of 
    the geometry.
    """
    depth = 8
    width = 0
    scale = 1.1
    
    poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd,
                                                                             depth,
                                                                             width,
                                                                             scale,
                                                                             linear_fit=False)[0]
    bbox = pcd.get_axis_aligned_bounding_box()
    cropped_poisson_mesh = poisson_mesh.crop(bbox)
    
    o3d.visualization.draw_geometries([cropped_poisson_mesh])
    
    return cropped_poisson_mesh

# cropped_poisson_mesh = poission_mesh(cl)

In [28]:
pm = poisson_mesh(pcd)

In [29]:
o3d.io.write_triangle_mesh("./test.off",
                           pm,
                           write_vertex_normals=False,
                           write_vertex_colors=False)

True

In [ ]:
def check_properties(name, mesh):
    mesh.compute_vertex_normals()

    edge_manifold = mesh.is_edge_manifold(allow_boundary_edges=True)
    edge_manifold_boundary = mesh.is_edge_manifold(allow_boundary_edges=False)
    vertex_manifold = mesh.is_vertex_manifold()
    self_intersecting = mesh.is_self_intersecting()
    watertight = mesh.is_watertight()
    orientable = mesh.is_orientable()

    print(name)
    print(f"  edge_manifold:          {edge_manifold}")
    print(f"  edge_manifold_boundary: {edge_manifold_boundary}")
    print(f"  vertex_manifold:        {vertex_manifold}")
    print(f"  self_intersecting:      {self_intersecting}")
    print(f"  watertight:             {watertight}")
    print(f"  orientable:             {orientable}")

In [ ]:
check_properties('Knot', bpa_mesh)

In [ ]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=25)
    bbox = pcd.get_axis_aligned_bounding_box()
#     mesh = mesh.crop(bbox)
print(mesh)
o3d.visualization.draw_geometries([mesh])


# **IDEA 1**: Cluster + Remove Outlier based on Average Radius

First check how many clusters are found in the data:
1. If cluster > 0:
    1. Compute average radius
    2. Remove outliers based on radius
    
2. If cluster == 0:
    1. No removal of points
    
**Note:**
1. For timeslice with only 3 points, it takes `eps` = 200 to identify. (default = 100)

In [16]:
def visualise(pcd):
    '''
    Visualise Point Cloud
    '''
    o3d.visualization.draw_geometries([pcd])

    
def count_clusters(pcd):
    '''
    Use DBScan to generate clusters
    '''
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(pcd.cluster_dbscan(eps=100, min_points=100, print_progress=True))
    
    max_label = labels.max() + 1
    print(f"point cloud has {max_label} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    
#     visualise(pcd)
    
    return max_label



def outlier_removal_validity(no_of_clusters):
    if no_of_clusters == 0:
        print("Save the whole thing & dont remove outliers")
        return False
    else:
        print("check for outliers and remove")  
        return True
    

def remove_outliers(pcd):
    '''
    Remove outliers based on radius
    '''
    
    distances = pcd.compute_nearest_neighbor_distance()
    avg_dist = np.mean(distances)
    radius = 4 * avg_dist
    print(radius)
    
    cl, ind = pcd.remove_radius_outlier(nb_points=32, radius=radius)
    return cl, ind


def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
    
def save_pointcloud(cl):
    '''
    Save the xyz points
    '''
    o3d.io.write_point_cloud("./test.xyz", cl)

In [17]:
#1 Find the number of clusters
no_of_clusters = count_clusters(pcd)

#2 Obtain validity status
validity = outlier_removal_validity(no_of_clusters)

#3 
if validity == True:
    cl, ind = remove_outliers(pcd)
    display_inlier_outlier(pcd, ind)
#     poisson_mesh(cl)
else:
#     poission_mesh(pcd)
    print("Nothing to do...yet!")
    
#4 Save xyz
# save_pointcloud(cl)

# visualise(cl)

[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 0
point cloud has 0 clusters
Save the whole thing & dont remove outliers
Nothing to do...yet!


# Use DBScan to Remove Outliers

In [7]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

X = np.array(pcd.points)
db = DBSCAN(eps=100, min_samples=100).fit(X)

In [ ]:
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
dbsc_clusters = pd.Series([X[labels==n] for n in  range(n_clusters_)])


In [ ]:
dbsc_clusters 

In [ ]:
def get_centroid(cluster):
    cluster_ary = np.asarray(cluster)
    centroid = cluster_ary.mean(axis=0)
    return centroid

# get centroid of each cluster
cluster_centroids = dbsc_clusters.map(get_centroid)
# unzip the list of centroid points (lat, lon) tuples into separate lat and lon lists
cents = zip(*cluster_centroids)
# # from these lats/lons create a new df of one representative point for eac   cluster
centroids_df = pd.DataFrame({'centroids':cents})

In [ ]:
cluster_centroids

In [ ]:
centroids_df

# Use 2.5D Delauney (Best Fitting Plane) + Laplacian Smoothing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.tri as mtri
from scipy.spatial import Delaunay
points = np.array([[0, 0], [0, 1.1], [1, 0], [1, 1]])


In [ ]:
type(points)

In [ ]:
u = np.array(pcd.points)[:,0]
v = np.array(pcd.points)[:,1]
z = np.array(pcd.points)[:,2]

x = u
y = v

tri = Delaunay(np.array([u, v]).T)
tri

In [ ]:
type(np.array(pcd.points))

In [ ]:
# print ('polyhedron(faces = [')
# #for vert in tri.triangles:
# for vert in tri.simplices:
#     print ('[%d,%d,%d],' % (vert[0],vert[1],vert[2]),'], points = [')
# for i in range(x.shape[0]):
#     print ('[%f,%f,%f],' % (x[i], y[i], z[i]),']);')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_trisurf(x,y,z,
           triangles=tri.simplices)


plt.show()

In [ ]:
tri.simplices

In [ ]:
CHECK ALGOs because thet are not very precise atm.